In [1]:
import tensorflow as tf
import os
import cv2
import numpy as np
# from gtts import gTTS
from playsound import playsound

# Loading the trained model

In [2]:
model = tf.keras.models.load_model("./models/version-3")

# Detect Eyes Using Webcam

In [3]:
face_cascade = cv2.CascadeClassifier("../Haarcascades/haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier("../Haarcascades/haarcascade_eye.xml")

In [4]:
# def speak(text, filename):
#     tts = gTTS(text=text,lang="en")
#     filepath = os.path.join("sounds", filename)
#     print(filepath)
#     tts.save(filepath)
#     playsound(filepath)


In [5]:
# speak("Wake UP!", "Wake_up.mp3")

In [6]:
# Helper function that detects faces and eyes in the given frame, and then predicts whether the eyes are open or closed
# if closed, increments count by 1, else sets it to 0.
def detect(gray, frame, count=0, renderFace=False, renderEyes=False, showLabels=False):
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    for (x,y,w,h) in faces:
        # Draw face
        if renderFace:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
            cv2.putText(frame, 'Face', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

        roi_gray=gray[y:y+h, x:x+w]
        roi_color=frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.1, 3)

        for (ex,ey,ew,eh) in eyes:
            buffer = 25
            startX = max(0, ex-buffer)
            startY = max(0, ey-buffer)
            endX = min(ex + ew + buffer,frame.shape[1])
            endY = min(ey + eh + buffer,frame.shape[0])
            if renderEyes:
                cv2.rectangle(roi_color, (startX,startY), (endX, endY), (255,0,0), 2) # Includes Buffer
            eyeImg = roi_color[startY:endY, startX:endX]  # Includes Buffer
            eyeImg=eyeImg/255.0
            eyeImg = cv2.resize(eyeImg,(256,256))
            expandedImg = np.expand_dims(eyeImg, axis=0)
            pred = model.predict(expandedImg)
            if showLabels:
                text = ("Drowsy" if pred[0][0] < 0.5 else "Not Drowsy") + str(pred[0][0])
                cv2.putText(roi_color, text, (ex, ey-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 1)
            count = (count + 1) if pred[0][0] < 0.5 else 0
    return frame, count

In [11]:
num_drowsy_frames=0
MIN_DROWSY_FRAMES=10
video = cv2.VideoCapture(1)

while True:
    _, frame = video.read()
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    canvas, num_drowsy_frames = detect(gray, 
                                       frame, 
                                       num_drowsy_frames,
                                       renderEyes=True, 
                                       showLabels=True)

    if(num_drowsy_frames>=MIN_DROWSY_FRAMES):
        playsound(os.path.join("sounds","Wake_up.mp3"), False)
        num_drowsy_frames=0
        
    cv2.imshow("Video", canvas)
    k=cv2.waitKey(1)
    if k & 0xFF == ord('q'):
        print("Quit")
        break
    if k%256==27:  # For Escape Key
        print("Escape")
        break
video.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


1/1 [==============================] - 0s 18ms/step
Quit


-1